In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import calendar
from pathos.multiprocessing import ProcessingPool as Pool
import openpyxl
#from multiprocessing import Pool
from time import sleep

cal = calendar.Calendar()

query = "남성"
year = 2019

def scrape(url):
    import requests
    try:
        res = requests.get(url, headers={'User-Agent':'Mozilla/5.0'}, allow_redirects=False, timeout=10)
        return res.text
    except:
        return "-"
    
if __name__ == '__main__':
    
    for month in range(5, 13):
        monthdays = [d for d in cal.itermonthdays(year, month) if d != 0]
        for day in monthdays:
            
            #start_time = time.time()
            
            date1 = str(year) + "." + str(month).zfill(2) + "." + str(day).zfill(2)
            date2 = date1.replace(".","")
            print(date1)

            xlsx_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', query, '#1_'+ str(year) + '_'+query, 
                                     '{}.xlsx'.format("#1_"+date2+"_"+query))
            wb = openpyxl.Workbook()
            sheet = wb.active
            sheet.append(["title", "source", "date", "html", "url"])
            wb.save(xlsx_name)
            
            book = openpyxl.load_workbook(xlsx_name)
            writer = pd.ExcelWriter(xlsx_name, engine='openpyxl')
            writer.book = book
            writer.sheets = {ws.title: ws for ws in book.worksheets}            

            for n in range(1, 4001, 10):
                url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=1&ds=" + date1 + "&de=" + date1 + "&nso=so%3Ar%2Cp%3Afrom" + date2 + "to" + date2 + "%2Ca%3A&start=" + str(n)
                raw = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
                html = BeautifulSoup(raw.text, "html.parser")
                articles = html.select("ul.type01 > li")

                if not articles:
                    break
                
                title, urls, source, date, records, html = list(), list(), list(), list(), list(), list()
                
                for ar in articles:
                    
                    title.append(ar.select_one("a._sp_each_title").text)
                    if ar.select_one("dd a._sp_each_url"):
                        urls.append(ar.select_one("dd a._sp_each_url").get('href'))
                    else:
                        urls.append(ar.select_one("a").get('href'))
                    source.append(ar.select_one("span._sp_each_source").text)
                    date.append(date2)
                
                with Pool(10) as p:
                    records = p.map(scrape, urls)
                
                for re in records:
                    html.append(BeautifulSoup(re, "lxml"))
                
                d = pd.DataFrame({"title": title, "source": source, 
                                  "date": date, "html": html, "url": urls}) 
                sleep(0.001)
                
                for sheetname in writer.sheets:
                    try:
                        d.to_excel(writer,sheet_name=sheetname, startrow=writer.sheets[sheetname].max_row, index = False,header= False)
                    except:
                        pass
                        
            writer.save()
                
            #print("--- %s seconds ---" % (time.time() - start_time))

2019.02.01
2019.02.02
2019.02.03
2019.02.04
2019.02.05
2019.02.06
2019.02.07
2019.02.08
2019.02.09
2019.02.10
2019.02.11
2019.02.12
2019.02.13
2019.02.14
2019.02.15
2019.02.16
2019.02.17
2019.02.18
2019.02.19
2019.02.20
2019.02.21
2019.02.22
2019.02.23
2019.02.24
2019.02.25
2019.02.26
2019.02.27
2019.02.28
2019.03.01
2019.03.02
2019.03.03
2019.03.04
2019.03.05
2019.03.06
2019.03.07
2019.03.08
2019.03.09
2019.03.10
2019.03.11
2019.03.12
2019.03.13
2019.03.14
2019.03.15
2019.03.16
2019.03.17
2019.03.18
2019.03.19
2019.03.20
2019.03.21
2019.03.22
2019.03.23
2019.03.24
2019.03.25
2019.03.26
2019.03.27
2019.03.28
2019.03.29
2019.03.30
2019.03.31
2019.04.01
2019.04.02
2019.04.03
2019.04.04
2019.04.05
2019.04.06
2019.04.07
2019.04.08
2019.04.09
2019.04.10
2019.04.11
2019.04.12
2019.04.13
2019.04.14
2019.04.15
2019.04.16
2019.04.17


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import calendar
from pathos.multiprocessing import ProcessingPool as Pool
import openpyxl
#from multiprocessing import Pool
from time import sleep

cal = calendar.Calendar()

query = "남성"
year = 2019
month = 4
s_date = 17

def scrape(url):
    import requests
    try:
        res = requests.get(url, headers={'User-Agent':'Mozilla/5.0'}, allow_redirects=False, timeout=10)
        return res.text
    except:
        return "-"
    
if __name__ == '__main__':
    
    for day in range(s_date, calendar.monthrange(year, month)[1]+1):

        date1 = str(year) + "." + str(month).zfill(2) + "." + str(day).zfill(2)
        date2 = date1.replace(".","")
        print(date1)

        xlsx_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', query, '#1_'+ str(year) + '_'+query, 
                                 '{}.xlsx'.format("#1_"+date2+"_"+query))
        wb = openpyxl.Workbook()
        sheet = wb.active
        sheet.append(["title", "source", "date", "html", "url"])
        wb.save(xlsx_name)

        book = openpyxl.load_workbook(xlsx_name)
        writer = pd.ExcelWriter(xlsx_name, engine='openpyxl')
        writer.book = book
        writer.sheets = {ws.title: ws for ws in book.worksheets}            

        for n in range(1, 4001, 10):
            url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=1&ds=" + date1 + "&de=" + date1 + "&nso=so%3Ar%2Cp%3Afrom" + date2 + "to" + date2 + "%2Ca%3A&start=" + str(n)
            raw = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
            html = BeautifulSoup(raw.text, "html.parser")
            articles = html.select("ul.type01 > li")

            if not articles:
                break

            title, urls, source, date, records, html = list(), list(), list(), list(), list(), list()

            for ar in articles:

                title.append(ar.select_one("a._sp_each_title").text)
                if ar.select_one("dd a._sp_each_url"):
                    urls.append(ar.select_one("dd a._sp_each_url").get('href'))
                else:
                    urls.append(ar.select_one("a").get('href'))
                source.append(ar.select_one("span._sp_each_source").text)
                date.append(date2)

            with Pool(10) as p:
                records = p.map(scrape, urls)

            for re in records:
                html.append(BeautifulSoup(re, "lxml"))

            d = pd.DataFrame({"title": title, "source": source, 
                              "date": date, "html": html, "url": urls}) 
            sleep(0.001)

            for sheetname in writer.sheets:
                try:
                    d.to_excel(writer,sheet_name=sheetname, startrow=writer.sheets[sheetname].max_row, index = False,header= False)
                except:
                    pass
                    
        writer.save()

2019.04.17
2019.04.18
2019.04.19
2019.04.20
2019.04.21
2019.04.22
2019.04.23
2019.04.24
2019.04.25
2019.04.26
2019.04.27
2019.04.28
2019.04.29
2019.04.30


In [ ]:
 raw = requests.get("https://news.naver.com/main/list.nhn?mode=LPOD&mid=sec&oid=032&listType=title&date="+str(date)+"&page="+str(page),
                           headers={"User-Agent":"Mozilla/5.0"})